<a href="https://colab.research.google.com/github/nermienkh/POC-Text-2-Text-Chatbot/blob/main/QA_with_Context.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers==4.19.2
!pip install pyarabic
!pip install arabert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 23.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186458 sha256=c9a02793274467be53c18c887fba6b2dd56c155dce1c2d0e0e76e9c7923551cf
  Stored in directory: /root/.cache/pip/wheels/10/f0/fd/4813b1177405693e8da9cdea839f0fb64fde161380e058c827
Successfully built emoji


In [17]:
!pwd

/content


In [7]:

from transformers import ElectraForQuestionAnswering, ElectraForSequenceClassification, AutoTokenizer, pipeline
from arabert.preprocess import ArabertPreprocessor


In [13]:
prep_object = ArabertPreprocessor(model_name="araelectra-base-discriminator")


In [27]:
question = prep_object.preprocess('إلى أين سيذهب رياض ؟')
context = prep_object.preprocess('''
رياض يستعد للمدرسة قام رياض من نومه , ترك فراشه وغسل يديه ووجهه , ثم لبس ثيابه وجلس يتناول فطوره . أكل رياض بيضة وشرب كأس حليب
''')
# a) Get predictions
qa_modelname = 'ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA'
cls_modelname = 'ZeyadAhmed/AraElectra-Arabic-SQuADv2-CLS'
qa_pipe = pipeline('question-answering', model=qa_modelname, tokenizer=qa_modelname)
QA_input = {
    'question': question,
    'context': context
}
CLS_input = {
    'text': question,
    'text_pair': context
}
qa_res = qa_pipe(QA_input)
#cls_res = cls_pipe(CLS_iput)
threshold = 0.5 #hyperparameter can be tweaked
## note classification results label0 probability it can be answered label1 probability can't be answered
## if label1 probability > threshold then consider the output of qa_res is empty string else take the qa_res
# b) Load model & tokenizer
qa_model = ElectraForQuestionAnswering.from_pretrained(qa_modelname)
cls_model = ElectraForSequenceClassification.from_pretrained(cls_modelname)
tokenizer = AutoTokenizer.from_pretrained(qa_modelname)

In [18]:
tokenizer.tokenize(prep_object.preprocess(question))

['ما', 'هي', 'جامعة', 'الدول', 'العربية', '؟']

In [28]:
qa_res

{'score': 0.08257149159908295, 'start': 11, 'end': 18, 'answer': 'للمدرسة'}